In [1]:
import numpy as np
# import cv2
import tifffile as tiff
from shapely.geometry import Polygon, mapping, shape, box
from skimage import io
from skimage.measure import label, regionprops
# import matplotlib.pyplot as plt
from rasterio import features
import scanpy as sc
import pandas as pd
import geopandas as gpd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import scipy

from SGanalysis.SGobject import SGobject 


from sklearn.cluster import AgglomerativeClustering as AggCluster

import sys
import os

tools_path  = '/Users/grantkinsler/Documents/Penn/Research/SpatialBarcodes/analysis/tools/tools.py'
# tools_path  = '/Users/grantkinsler/Documents/Penn/Research/SpatialBarcodes/SpatialBarcodes/analysis/tools/tools.py'
# tools_path = '/home/wniu/Documents/GRK/SpatialBarcodes/analysis/tools/tools.py'
sys.path.append(os.path.dirname(os.path.expanduser(tools_path)))
import tools

In [3]:
### roi_name: [segmentation_file, spots_file, out_path]
roi_file_paths = {
    # 'roi_1':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/segmentations/nuclei_20240604_nuclei.tiff',
    #                        'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/exports/decode_20240604.csv',
    #                         'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/exports'
    #                        },
    #               'roi_2':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/segmentations/nuclei_20240529_nuclei.tiff',
    #                        'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports/decode_20240604.csv',
    #                         'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports'
    #                        },
    #               'roi_3':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/segmentations/nuclei_20240604_nuclei.tiff',
    #                        'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/exports/decode_20240604.csv',
    #                         'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/exports'
    #                        },
                #   'timezero_roi_1':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/segmentations/segmentation_20240513_nuclei.tiff',
                #               # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                #               'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/transcripts/20240517_segmentation_withRefid.csv',
                #               'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/exports'
                #               },
                    # 'timezero_roi_2':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi2/segmentations/segment_091924_nuclei.tiff',
                    #           # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                    #           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi2/transcripts/transcripts.csv',
                    #           'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi2/exports'
                    #           },
                    # 'timezero_roi_3':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/segmentations/segment_20240921_nuclei.tiff',
                    #           # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                    #           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/transcripts/transcripts.csv',
                    #           'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/exports'
                    #           },


                    #  'run2_roi_1':{'segmentation_file':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_1/exports/segmentation.tiff',
                    #           'spots_file':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_1/exports/transcripts.csv',
                    #            'out_path':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_1/exports/',
                    #             },
#                    'run2_roi_2':{'segmentation_file':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_2/exports/segmentation.tiff',
#                                'spots_file':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_2/exports/transcripts.csv',
#                                'out_path':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_2/exports/',
#                                },
#                    'run2_roi_3':{'segmentation_file':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_3/exports/segmentation.tiff',
#                                'spots_file':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_3/exports/transcripts.csv',
#                                'out_path':'/media/wniu/GRK_003/2024-08-08_spatialbarcode_tumor2_projects/roi_3/exports/',
#                                },
                    # 'dish_roi1':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_1/segmentations/segmentation_1_nuclei.tiff',
                    #           # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                    #           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_1/exports/transcripts.csv',
                    #           'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_1/exports'
                    #           },
        
                    'dish_roi2':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_2/segmentations/segmentation_1_nuclei.tiff',
                              # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                              'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_2/exports/transcripts.csv',
                              'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_2/exports/'
                              },

                    # 'dish_roi3':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_3/segmentations/20240321_segmentation_2_cells.tiff',
                    #           # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                    #           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_3/exports/transcripts.csv',
                    #           'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_3/exports/'
                    #           },
                  
                  }


In [ ]:
partially_processed = False
cutoff = 3
barcode_cols = ['bc_{:03d}'.format(i) for i in range(1,97)]

from SGanalysis.SGobject import SGobject

for roi_name,files in roi_file_paths.items():

    print(f'Processing {roi_name}...')

    segmentation_file = files['segmentation_file']
    spots_file = files['spots_file']
    out_path = files['out_path']
    
    if not partially_processed:
        # Create an instance of SGobject
        sg_obj = SGobject()

        # Convert a TIFF image to polygons and store them in a GeoDataFrame
        print("Running mask_to_objects...")
        sg_obj.mask_to_objects(segmentation_file)

        sg_obj.load_points(spots_file)

        sg_obj.dilate_objects(10)

        ## associate spots with segmentation

        sg_obj.create_cell_gene_table()

        # with open(f'{out_path}/sg_object_dilate10_20240718.pkl', 'wb') as f:
        #     pickle.dump(sg_obj, f)
            
        matrix = sg_obj.get_cell_gene_table_df()
        matrix['object_id'] = [str(int(x)) for x in matrix.index]
        matrix.set_index('object_id',inplace=True)

        sg_obj.gdf['object_id'] = [str(int(x)) for x in sg_obj.gdf['object_id']]
        sg_obj.gdf.set_index('object_id',inplace=True)

        sg_obj.gdf['nucleus_centroid'] = sg_obj.gdf['nucleus'].centroid.values
        sg_obj.gdf['center_x'] = sg_obj.gdf['nucleus'].centroid.x.values
        sg_obj.gdf['center_y'] = sg_obj.gdf['nucleus'].centroid.y.values
        sg_obj.gdf['area'] = sg_obj.gdf['nucleus_dilated'].area.values

        barcode_cols = ['bc_{:03d}'.format(i) for i in range(1,97)]


        ## traditional barcode calling (using cutoff of 3

        cell_barcodes = {}
        # matrix.set_index('object_id',   inplace=True)
        # df.set_index('object_id')

        df = matrix
        df['cell_id'] = df.index

        for cell_id in df['cell_id']:
            this_cell = df[df['cell_id']==cell_id]
            cell_barcodes[cell_id] = []

            for bc in barcode_cols:
                if this_cell[bc].values[0] >= cutoff:
                    cell_barcodes[cell_id].append(bc)

        df['called_barcodes'] = cell_barcodes.values()
        df['n_called_barcodes'] = [len(bc_set) for bc_set in cell_barcodes.values()]
        df['barcode_names'] = ['-'.join(sorted(bc_set)) for bc_set in cell_barcodes.values()]

        df = pd.merge(df,sg_obj.gdf,how='left',left_index=True,right_index=True)

Processing dish_roi2...
Running mask_to_objects...
Dilation completed. Output column: 'nucleus_dilated' is now the active geometry column.
6508061 of 12227072 spots (53.2%) assigned to an object.


/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/utils.py:143: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/SGanalysis/SGobject.py:217: FutureWarning: X.dtype being converted to np.float32 from object. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self.cell_gene_table = sc.AnnData(pivot_table_full)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


No filtering available or applied.


In [25]:
sg_obj = SGobject()

# Convert a TIFF image to polygons and store them in a GeoDataFrame
print("Running mask_to_objects...")
sg_obj.mask_to_objects(segmentation_file)

sg_obj.load_points(spots_file)

sg_obj.dilate_objects(10)

## associate spots with segmentation

sg_obj.create_cell_gene_table()

Running mask_to_objects...
Dilation completed. Output column: 'nucleus_dilated' is now the active geometry column.
6508061 of 12227072 spots (53.2%) assigned to an object.


/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/utils.py:143: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/SGanalysis/SGobject.py:217: FutureWarning: X.dtype being converted to np.float32 from object. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self.cell_gene_table = sc.AnnData(pivot_table_full)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [26]:
matrix = sg_obj.get_cell_gene_table_df()

No filtering available or applied.


In [27]:
matrix

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,bc_087,bc_088,bc_089,bc_090,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096
0,35.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,7.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,29.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,16364.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
16372,16375.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16373,16361.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,6.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16374,16357.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
sg_obj.gdf

,object_id,nucleus,nucleus_dilated
0,35,"POLYGON ((9279.000 2.000, 9349.000 3.000, 9356...","POLYGON ((9276.253 -7.615, 9269.253 -5.615, 92..."
1,34,"POLYGON ((6437.000 2.000, 6520.000 2.000, 6527...","POLYGON ((6435.678 -7.912, 6420.678 -5.912, 64..."
2,29,"POLYGON ((24636.000 0.000, 24762.000 0.000, 24...","POLYGON ((24628.611 -6.720, 24624.039 -5.806, ..."
3,11,"POLYGON ((11474.000 0.000, 11542.000 0.000, 11...","POLYGON ((11466.611 -6.720, 11462.039 -5.806, ..."
4,15,"POLYGON ((14595.000 0.000, 14678.000 2.000, 14...","POLYGON ((14549.240 13.160, 14549.489 14.089, ..."
...,...,...,...
16371,16364,"POLYGON ((19896.000 35571.000, 19914.000 35571...","POLYGON ((19833.086 35616.214, 19833.656 35616..."
16372,16375,"POLYGON ((20380.000 35592.000, 20408.000 35596...","POLYGON ((20336.126 35615.151, 20336.270 35615..."
16373,16361,"POLYGON ((22881.000 35565.000, 22904.000 35566...","POLYGON ((22808.307 35615.384, 22808.929 35616..."
16374,16357,"POLYGON ((24819.000 35560.000, 24840.000 35560...","POLYGON ((24726.585 35615.691, 24726.929 35616..."


In [15]:
matrix

,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,[bc_066],1,bc_066
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1,"[bc_019, bc_030, bc_073]",3,bc_019-bc_030-bc_073
2,4.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,16.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,"[bc_014, bc_078]",2,bc_014-bc_078
3,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,[bc_027],1,bc_027
4,3.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,23.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,"[bc_018, bc_060]",2,bc_018-bc_060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,2.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0,13.0,1.0,...,0.0,0.0,0.0,0.0,8.0,0.0,16371,"[bc_035, bc_057, bc_066, bc_095]",4,bc_035-bc_057-bc_066-bc_095
16372,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16372,[],0,
16373,2.0,0.0,0.0,0.0,2.0,0.0,0.0,6.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16373,"[bc_019, bc_030, bc_085]",3,bc_019-bc_030-bc_085


In [21]:
sg_obj.gdf['object_id_int'] = [int(x) for x in sg_obj.gdf.index]

In [23]:
sg_obj.gdf.sort_values('object_id_int')

,nucleus,nucleus_dilated,nucleus_centroid,center_x,center_y,area,cell_id,object_id_int
object_id,,,,,,,,
1,"POLYGON ((1250.000 0.000, 1333.000 0.000, 1333...","POLYGON ((1242.477 -6.572, 1238.831 -5.762, 12...",POINT (1294.255 52.400),1294.255129,52.399954,16692.990255,1,1
2,"POLYGON ((1647.000 0.000, 1741.000 0.000, 1741...","POLYGON ((1641.020 -8.000, 1637.000 -8.000, 16...",POINT (1694.213 45.568),1694.213299,45.568080,17156.994367,2,2
3,"POLYGON ((1871.000 0.000, 1891.000 0.000, 1891...","POLYGON ((1865.020 -8.000, 1840.000 -8.000, 18...",POINT (1859.179 29.849),1859.179108,29.849034,6962.736388,3,3
4,"POLYGON ((2584.000 0.000, 2660.000 0.000, 2660...","POLYGON ((2576.307 -6.384, 2573.575 -5.701, 25...",POINT (2630.961 53.509),2630.961345,53.508664,17825.724642,4,4
5,"POLYGON ((5020.000 0.000, 5100.000 0.000, 5100...","POLYGON ((5012.885 -7.023, 5005.479 -5.884, 50...",POINT (5052.133 21.586),5052.132638,21.586448,7032.348119,5,5
...,...,...,...,...,...,...,...,...
16372,"POLYGON ((20563.000 35587.000, 20609.000 35589...","POLYGON ((20517.720 35615.840, 20517.929 35616...",POINT (20576.835 35599.712),20576.835153,35599.711720,4954.123267,16372,16372
16373,"POLYGON ((9597.000 35588.000, 9633.000 35590.0...","POLYGON ((9563.896 35614.840, 9564.270 35615.3...",POINT (9613.633 35600.156),9613.632838,35600.155785,3921.206502,16373,16373
16374,"POLYGON ((20120.000 35591.000, 20145.000 35591...","POLYGON ((20084.139 35616.262, 20084.656 35616...",POINT (20130.122 35601.147),20130.121861,35601.147201,4022.606267,16374,16374


In [13]:
sg_obj.gdf['cell_id'] = sg_obj.gdf.index

In [14]:
cell_id = 167

region = tools.get_polygons_around_polygon(sg_obj,cell_id,image_scale=25,id_field='cell_id')

gene_names = tools.get_all_barcodes_in_region(df,region['cell_id'].values,id_field='cell_id')

fig = plt.figure(figsize=(10,10))

vmin = 0
# vmax = max([cell_gene_table_df[gene_of_interest].quantile(0.99),5])
vmax = max([df[gene_of_interest].quantile(0.99),10])

ax = fig.add_subplot(221)
tools.plot_polygons_and_points(sg_obj,[cell_id],gene_names=gene_names,
                            annotate=False,image_scale=0.5,label=cell_id,
                            focal_outline_color='k',other_outline_color='gray',
                            interior_marker='.',single_mode=False,lw=0.5,
                            exterior_marker='.',marker_size=1,ax=ax)

#     plt.title(f'Cluster {cluster_of_interest} barcodes')

#     ymin,ymax = plt.ylim()
#     xmin,xmax = plt.xlim()

#     ax = fig.add_subplot(223)

#     merged_gdf.plot(column=gene_of_interest, cmap='Blues', legend=False,vmin=vmin,vmax=vmax,ax=ax)

#     rect = matplotlib.patches.Rectangle((xmin, ymin), (xmax-xmin), (ymax-ymin), linewidth=1, edgecolor='k', facecolor='none')

#     # Add the patch to the Axes
#     ax.add_patch(rect)

#     ax = fig.add_subplot(224)

#     norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

# # ColorbarBase derives from ScalarMappable and puts a colorbar
# # in a specified axes, so it has everything needed for a
# # standalone colorbar.  There are many more kwargs, but the
# # following gives a basic continuous colorbar with ticks
# # and labels.
#     cb1 = matplotlib.colorbar.ColorbarBase(ax, cmap='Blues',
#                                 norm=norm,
#                                 orientation='horizontal')
#     cb1.set_label('Spots per cell')
    
#     ax = fig.add_subplot(222)

#     gene_names = [gene_of_interest]

#     # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

#     # tools.plot_polygons_and_points(sg_obj,cluster_cells,gene_names=gene_names,central_polygon_ix=central_ix,
#     #                             annotate=False,image_scale=0.5,label=f'in cluster {clusters_of_interest[0]}',
#     #                             focal_outline_color='r',other_outline_color='gray',
#     #                             interior_marker='.',single_mode=False,lw=0.5,
#     #                             exterior_marker='.',marker_size=2,ax=ax)

#     plt.title(f'Cluster {cluster_of_interest} - {gene_of_interest}\n,Rank:{rank+1} Avg. expression:{cluster_mean:.2f}')

#     # vmin = 0
#     # vmax = max([cell_gene_table_df[gene_of_interest].quantile(0.99),5])
#     # vmax = max([cell_gene_table_df[gene_of_interest].quantile(0.99),10])
#     print('VRange',vmin,vmax)

#     # other_clusters = adata_genes[~adata_genes.obs['bc_cluster'].isin(clusters_of_interest)]

#     bbox = box(xmin,ymin,xmax,ymax)

#     other_clusters = sg_obj.gdf[sg_obj.gdf.geometry.intersects(bbox) & (~sg_obj.gdf['cell_id'].isin(cluster_cells))]

#     merged_gdf_other = merged_gdf[merged_gdf['cell_id'].isin(other_clusters['cell_id'].values)]

#     # print(merged_gdf_focal.shape)

#     merged_gdf_other.plot(column=gene_of_interest, cmap='Blues', legend=False, ax=ax,
#                     vmin=vmin,vmax=vmax,edgecolor='gray',linewidth=0.05)

#     this_cluster = adata_genes[adata_genes.obs['bc_cluster']==cluster_of_interest]

#     merged_gdf_focal = merged_gdf[merged_gdf['cell_id'].isin(this_cluster.obs['cell_id'].values)]
    
#     # print(merged_gdf_focal.shape)
#     merged_gdf_focal.plot(column=gene_of_interest, cmap='Blues', legend=False, ax=ax,
#                     vmin=vmin,vmax=vmax,edgecolor='r',linewidth=0.5)

#     plt.xlim(xmin,xmax)
#     plt.ylim(ymin,ymax)

#     plt.savefig(f'plots/figures/cluster_intrinsic_expression_{gene_of_interest}_{cluster_of_interest}.pdf',bbox_inches='tight')
#     # sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False)

IndexError: single positional indexer is out-of-bounds